In [0]:
import tecton 
import pandas

from datetime import timedelta, datetime
from tecton import Entity, BatchSource, batch_feature_view, spark_batch_config, Aggregation, RequestSource, on_demand_feature_view
from tecton.types import Float64, Field, String

tecton.set_credentials(tecton_api_key=dbutils.secrets.get(scope='tecton-lab-2', key='TECTON_API_KEY'),tecton_url="https://lab.tecton.ai/api")
ws = tecton.get_workspace('prod')

In [0]:
rows = [
  {
    "user_id": "12345",
    "payment_id": "54321",
    "payment_date": datetime(2023, 7, 25),
    "payment_due_date": datetime(2023, 7, 20),
    "update_timestamp": datetime(2023, 7, 25),
    "amt": 10,
  },
  {
    "user_id": "12345",
    "payment_id": "54320",
    "payment_date": datetime(2023, 7, 25),
    "payment_due_date": datetime(2023, 7, 25),
    "update_timestamp": datetime(2023, 7, 25),
    "amt": 9,
  },
  {
    "user_id": "12345",
    "payment_id": "54310",
    "payment_date": datetime(2023, 7, 25),
    "payment_due_date": datetime(2023, 7, 25),
    "update_timestamp": datetime(2023, 7, 25),
    "amt": 8,
  },
  {
    "user_id": "10000",
    "payment_id": "10101",
    "payment_date": datetime(2023, 7, 25),
    "payment_due_date": datetime(2023, 7, 15),   
    "update_timestamp": datetime(2023, 7, 25),
    "amt": 7,
  },
  {
    "user_id": "10000",
    "payment_id": "10102",
    "payment_date": datetime(2023, 7, 25),
    "payment_due_date": datetime(2023, 7, 10),
    "update_timestamp": datetime(2023, 7, 25),
    "amt": 6,
  },
  {
    "user_id": "10000",
    "payment_id": "10103",
    "payment_date": datetime(2023, 7, 25),
    "payment_due_date": datetime(2023, 7, 5),
    "update_timestamp": datetime(2023, 7, 25),
    "amt": 5,
  },
]

df_spark = spark.createDataFrame(pandas.DataFrame.from_records(rows))
display(df_spark)

user_id,payment_id,payment_date,payment_due_date,update_timestamp,amt
12345,54321,2023-07-25T00:00:00.000+0000,2023-07-20T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,10
12345,54320,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,9
12345,54310,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,8
10000,10101,2023-07-25T00:00:00.000+0000,2023-07-15T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,7
10000,10102,2023-07-25T00:00:00.000+0000,2023-07-10T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,6
10000,10103,2023-07-25T00:00:00.000+0000,2023-07-05T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,5


In [0]:
df_spark.createOrReplaceTempView("data")

In [0]:
@spark_batch_config()
def batch_config(spark):
  return df_spark

late_payments_source = BatchSource(
    name='late_payments_source',
    batch_config=batch_config
)

late_payments_source.validate()
display(late_payments_source.get_dataframe().to_spark())

BatchSource 'late_payments_source': Deriving schema.
BatchSource 'late_payments_source': Successfully validated.

user_id,payment_id,payment_date,payment_due_date,update_timestamp,amt
12345,54321,2023-07-25T00:00:00.000+0000,2023-07-20T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,10
12345,54320,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,9
12345,54310,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,8
10000,10101,2023-07-25T00:00:00.000+0000,2023-07-15T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,7
10000,10102,2023-07-25T00:00:00.000+0000,2023-07-10T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,6
10000,10103,2023-07-25T00:00:00.000+0000,2023-07-05T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,5


In [0]:
user = Entity(
    name="user",
    join_keys=["user_id"],
)
user.validate()

@batch_feature_view(
    sources=[late_payments_source],
    entities=[user],
    mode='spark_sql',
    batch_schedule=timedelta(days=1),
    feature_start_time=datetime(2023,7,1),
    timestamp_field="update_timestamp"
)
def features(late_payments_source):
  return f'''
    select user_id, 
    payment_id,
    amt, 
    update_timestamp
    from {late_payments_source}
  '''
features.validate()

Entity 'user': Successfully validated.
BatchFeatureView 'features': Validating 1 of 3 dependencies. (2 already validated)
 Transformation 'features': Successfully validated.
BatchFeatureView 'features': Successfully validated.

In [0]:
@batch_feature_view(
    sources=[late_payments_source],
    entities=[user],
    mode='spark_sql',
    batch_schedule=timedelta(days=1),
    aggregation_interval=timedelta(days=1),
    aggregations=[
        Aggregation(column='late_payment', function='sum', time_window=timedelta(days=365*10)),
        Aggregation(column='payment_id', function='count', time_window=timedelta(days=365*10)),
    ],
    feature_start_time=datetime(2023,7,1),
    timestamp_field="update_timestamp"
)
def payment_aggregates(late_payments_source):
  return f'''
    select user_id, 
        payment_id, 
        CASE when datediff(payment_date, payment_due_date) > 0 THEN 1 ELSE 0 END as late_payment,
        update_timestamp
    from {late_payments_source}
  '''
payment_aggregates.validate()

BatchFeatureView 'payment_aggregates': Validating 1 of 3 dependencies. (2 already validated)
 Transformation 'payment_aggregates': Successfully validated.
BatchFeatureView 'payment_aggregates': Successfully validated.

In [0]:
%sql
select *
from data

user_id,payment_id,payment_date,payment_due_date,update_timestamp,amt
12345,54321,2023-07-25T00:00:00.000+0000,2023-07-20T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,10
12345,54320,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,9
12345,54310,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,8
10000,10101,2023-07-25T00:00:00.000+0000,2023-07-15T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,7
10000,10102,2023-07-25T00:00:00.000+0000,2023-07-10T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,6
10000,10103,2023-07-25T00:00:00.000+0000,2023-07-05T00:00:00.000+0000,2023-07-25T00:00:00.000+0000,5


In [0]:
%sql
select user_id, payment_id, CASE when datediff(payment_date, payment_due_date) > 0 THEN 1 ELSE 0 END as late_payment
from data

user_id,payment_id,late_payment
12345,54321,1
12345,54320,0
12345,54310,0
10000,10101,1
10000,10102,1
10000,10103,1


In [0]:
z = payment_aggregates.get_historical_features(start_time=datetime(2023, 1, 1), end_time=datetime(2023, 8, 1)).to_spark()
display(payment_aggregates.get_historical_features(start_time=datetime(2023, 1, 1), end_time=datetime(2023, 8, 1)).to_spark())

The provided start_time (2023-01-01T00:00:00+00:00) is before "payment_aggregates"'s feature_start_time (2023-07-01T00:00:00+00:00). No feature values will be returned before the feature_start_time.
The provided start_time (2023-01-01T00:00:00+00:00) is before "payment_aggregates"'s feature_start_time (2023-07-01T00:00:00+00:00). No feature values will be returned before the feature_start_time.

user_id,late_payment_sum_3650d_1d,payment_id_count_3650d_1d,update_timestamp,_effective_timestamp
10000,3,3,2023-07-26T00:00:00.000+0000,2023-07-26T00:00:00.000+0000
12345,1,3,2023-07-26T00:00:00.000+0000,2023-07-26T00:00:00.000+0000


In [0]:
request_schema = [Field("payment_timestamp", String),Field("payment_due_timestamp", String)]
transaction_request = RequestSource(schema=request_schema)
output_schema = [Field("late_payment_ratio", Float64)]

@on_demand_feature_view(
   #fill in with parameters described above
   sources=[transaction_request,payment_aggregates],
   mode='python',
   schema=output_schema

)
def late_ratio(transaction_request, payment_aggregates):
    #definition goes here
    payments = 0 if payment_aggregates['payment_id_count_3650d_1d'] is None else payment_aggregates['payment_id_count_3650d_1d']
    late_payments = 0 if payment_aggregates['late_payment_sum_3650d_1d'] is None else payment_aggregates['late_payment_sum_3650d_1d']

    return {'late_ratio': (late_payments+(datetime.strptime(transaction_request['payment_timestamp'],'%Y-%m-%dT%H:%M:%S.000+0000') > datetime.strptime(transaction_request['payment_due_timestamp'],'%Y-%m-%dT%H:%M:%S.000+0000')))/(1 + payments)}

late_ratio.validate()

OnDemandFeatureView 'late_ratio': Validating 1 of 2 dependencies. (1 already validated)
 Transformation 'late_ratio': Successfully validated.
OnDemandFeatureView 'late_ratio': Successfully validated.

In [0]:
x = "2023-07-25T00:00:00.000+0000"
y = "2023-07-20T00:00:00.000+0000"

In [0]:
late_ratio.run(transaction_request={"payment_timestamp":x,"payment_due_timestamp":y},payment_aggregates=z['late_payment_sum_3650d_1d', 'payment_id_count_3650d_1d'])

Out[70]: {'late_ratio': 0 1.0
 1 0.5
 dtype: float64}